In [58]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%load_ext autotime
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))
from difflib import SequenceMatcher
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import AffinityPropagation
import distance
import random
from itertools import repeat 
from sklearn.cluster import DBSCAN
from numpy import nan

import utils



The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.3 ms (started: 2021-08-31 14:34:08 +02:00)


In [16]:
Data_file = "~/Downloads/data_avec_erreurs_wasserstein.csv"
df = pd.read_csv(Data_file)

time: 427 ms (started: 2021-08-31 14:23:36 +02:00)


/Users/abaach/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14,15,16,23,24,25,26,28,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# AffinityPropagation: 

Perform Affinity Propagation Clustering of data.

dampingfloat, default=0.5

Damping factor (between 0.5 and 1)  is the extent to which the current value 
is maintained relative to incoming values (weighted 1 - damping). 
This in order to avoid numerical oscillations when updating these values (messages)

Affinity is the distance that u need to use. 

In [64]:
df_State = np.asarray(df["UsageBand"])


AttributeError: 'numpy.ndarray' object has no attribute 'isnull'

time: 6 ms (started: 2021-08-31 14:34:55 +02:00)


# do not errase

In [65]:
df["UsageBand"][df["UsageBand"].notna()]


0           Low
1           Low
2          High
3          High
4        Medium
          ...  
99770    Medium
99779      High
99791    Medium
99919    Medium
99964    Medium
Name: UsageBand, Length: 23383, dtype: object

time: 7.05 ms (started: 2021-08-31 14:36:28 +02:00)


# type(np.asarray(df["UsageBand"])[-1])

In [73]:
def uncorrect_grammar(df_names, cluster):
    words = np.asarray(df_names)
    unique_words, unique_counts = np.unique(df_names, return_counts = True)
    Q_1 = round(np.percentile(unique_counts, 5))
    Low_risk_words = unique_words[np.where(unique_counts > Q_1)[0]]
    index_In_words = []
    for w in cluster:
        if not (w in list(Low_risk_words)):
            index_In_words = index_In_words + np.ndarray.tolist(np.where(words == w)[0])
    return index_In_words

time: 592 µs (started: 2021-08-31 14:41:56 +02:00)


In [85]:
df_State = df["UsageBand"][df["UsageBand"].notna()]
df_State_unique = np.unique(df_State)
words = np.asarray(df_State_unique) #So that indexing with a list will work
lev_similarity = np.array([[SequenceMatcher(None, w1, w2).ratio() for w1 in words] for w2 in words])

affprop = AffinityPropagation(affinity = "precomputed", damping=0.5)
affprop.fit(lev_similarity)
list_uncorrect = []
if len(np.unique(affprop.labels_)) == 1:
    list_uncorrect = []
else:
    for cluster_id in np.unique(affprop.labels_):
        cluster = np.unique(words[np.nonzero(affprop.labels_ == cluster_id)])
        list_uncorrect = list_uncorrect + uncorrect_grammar(df_State, cluster)
        print(cluster)


['HIGH' 'High' 'high']
['LOW' 'Low' 'low']
['MEDIUM' 'Medium' 'medium']
time: 57.2 ms (started: 2021-08-31 14:52:18 +02:00)


/Users/abaach/opt/anaconda3/lib/python3.8/site-packages/sklearn/cluster/_affinity_propagation.py:146: FutureWarning: 'random_state' has been introduced in 0.23. It will be set to None starting from 0.25 which means that results will differ at every function call. Set 'random_state' to None to silence this warning, or to 0 to keep the behavior of versions <0.23.
  warnings.warn(("'random_state' has been introduced in 0.23. "


In [86]:
 df_State[list_uncorrect]

2903       High
4782        Low
18318    Medium
19534    Medium
Name: UsageBand, dtype: object

time: 3.38 ms (started: 2021-08-31 14:52:20 +02:00)
